In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling

try:
    from peft import prepare_model_for_int8_training
except ImportError:
    from peft import prepare_model_for_kbit_training as prepare_model_for_int8_training

# 导入 LoRA 相关函数
from peft import LoraConfig, get_peft_model

# 导入 Hugging Face datasets 库中的 Dataset
from datasets import Dataset

# 尝试从 TRL 导入 SFTTrainer 和 SFTTrainingArguments，否则回退使用 Transformers 的 Trainer 和 TrainingArguments
try:
    from trl import SFTTrainer, SFTTrainingArguments
    training_args_class = SFTTrainingArguments
    trainer_class = SFTTrainer
except ImportError:
    from transformers import TrainingArguments, Trainer
    training_args_class = TrainingArguments
    trainer_class = Trainer

# 定义模型 checkpoint（使用 SmolLM-1.7B-Instruct 模型）
model_name = "HuggingFaceTB/SmolLM-1.7B-Instruct"

# 加载 tokenizer，确保 add_special_tokens=True（默认开启）
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# 加载模型，采用 8-bit 模式和自动设备映射（帮助在 24GB VRAM 内运行）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

# 准备模型以进行 int8（或 k-bit）训练（用于 QLoRA）
model = prepare_model_for_int8_training(model)

# 定义 QLoRA 配置（可根据需要调整 target_modules）
lora_config = LoraConfig(
    r=16,                # LoRA rank
    lora_alpha=32,       # 缩放因子
    target_modules=["q_proj", "v_proj"],  # 对这些层应用 LoRA（如有需要可调整）
    lora_dropout=0.1,
    bias="none"
)

# 用 LoRA adapter 包装模型
model = get_peft_model(model, lora_config)

# 创建包含多样化对话示例的自定义数据集，
# 每个示例都在末尾显式包含 EOS token（通过 add_special_tokens=True 自动添加）
data = [
    {"text": "User: Who are you?\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: What is your name?\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: Please introduce yourself.\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: Could you tell me who you are?\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: Tell me about yourself.\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: Who is speaking?\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: May I know who you are?\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: What's your name?\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: Introduce yourself.\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: State your identity.\nAssistant: I am Alex, your personal AI researcher."},
    {"text": "User: Who am I talking to?\nAssistant: I am Alex, your personal AI researcher."}
]

# 转换为 Hugging Face Dataset
dataset = Dataset.from_list(data)

# 定义 tokenization 函数，设置 add_special_tokens=True 确保特殊 token 被添加
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=512, add_special_tokens=True)

# 对数据集进行 tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)

# 创建用于因果语言建模的数据整理器（会自动通过 shift input_ids 创建 labels）
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 设置训练参数（演示目的下禁用模型保存）
training_args = training_args_class(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=30,
    logging_steps=100,
    save_total_limit=0,  # 禁用保存
)

# 初始化 Trainer
trainer = trainer_class(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 开始训练（短时间的微调演示）
trainer.train()

# ----- 测试微调后的模型 -----
# 设置测试提示，此处 add_special_tokens=True 也确保 EOS token 会被添加
test_prompt = "User: Who are you?\nAssistant:"
print("Test Prompt:")
print(test_prompt)

# 获取模型所在设备
device = next(model.parameters()).device

# 对测试提示进行 tokenization
inputs = tokenizer(test_prompt, return_tensors="pt", add_special_tokens=True).to(device)

# 生成并打印 10 个多样化的响应（使用采样，依靠 eos_token_id 自动终止生成）
for i in range(10):
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50, 
        do_sample=True,       # 启用采样以获得多样化输出
        temperature=0.7,       # 控制随机性
        top_p=0.9,             # nucleus 采样
        eos_token_id=tokenizer.eos_token_id  # 显式设置 EOS token id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nGenerated Response {i+1}:")
    print(response)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

/tmp/ipykernel_1203/4149714402.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = trainer_class(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: 

Step,Training Loss
100,2.003200


Test Prompt:
User: Who are you?
Assistant:


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



Generated Response 1:
User: Who are you?
Assistant: I am Alex, your personal AI researcher. How can I assist you today?

User: What is the meaning of life?

Assistant: Ah, a question that has puzzled humans for centuries! The meaning of life is a complex

Generated Response 2:
User: Who are you?
Assistant: I am Alex, your personal AI researcher. How can I assist you today?

User: Hi, I'm trying to understand more about artificial intelligence. What is AI?

Alex: AI stands for Artificial Intelligence, which refers to the

Generated Response 3:
User: Who are you?
Assistant: I am your personal AI researcher, Alex. I'll be happy to help you with your research.

**Conversation:**

User: Hi Alex. I'm studying the concept of artificial general intelligence (AGI). Can you tell

Generated Response 4:
User: Who are you?
Assistant: I am Alex, your personal AI researcher. I will be conducting an experiment to explore the concept of self.

Dr. Rachel Kim: Good morning, Alex. I'm excited to be part